In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

net = nn.Sequential(
    nn.Linear(4, 256),  
    nn.ELU(),
    nn.Linear(256, 128),
    nn.ELU(),
    nn.Linear(128, 64),
    nn.ELU(),
    nn.Linear(64, 1)  
)

optimizer = optim.Adam(net.parameters(), lr=0.001)  
criterion = nn.MSELoss()

input_data = df_cleaned[['circuitId', 'raceId', 'points', 'wins']].values
target_data = df_cleaned['stop'].values

scaler = StandardScaler()
input_data_scaled = scaler.fit_transform(input_data)

X_train, X_test, y_train, y_test = train_test_split(input_data_scaled, target_data, test_size=0.3, random_state=42)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1).to(device)

train_data = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)

net.to(device)

best_loss = float('inf')
early_stop_counter = 0
patience = 20  

for epoch in range(1000):
    net.train() 
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()

        pred = net(X_batch)  
        loss = criterion(pred, y_batch) 

        loss.backward()  
        optimizer.step() 

    net.eval()  
    with torch.no_grad():
        y_test_pred = net(X_test_tensor)  
        val_loss = criterion(y_test_pred, y_test_tensor)  
    
    if val_loss.item() < best_loss:
        best_loss = val_loss.item()
        early_stop_counter = 0  
    else:
        early_stop_counter += 1
        if early_stop_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break

    if (epoch+1) % 10 == 0:
        print(f'Epoch {epoch+1}, Training Loss: {loss.item()}, Validation Loss: {val_loss.item()}')

net.eval()
with torch.no_grad():
    y_test_pred = net(X_test_tensor)
    final_test_loss = criterion(y_test_pred, y_test_tensor)
    print(f'\nFinal Test Loss: {final_test_loss.item()}')
